## Import Libraries and Functions and Define the Model

In [1]:
# Physics Informed Neural Network with Taylor Series

from utils import *
import math
import torch.nn as nn

'''
N_INPUT: The number of input bio-z dimensions for one heartbeat
N_FEAT: The number of physiological features
N_EXT: The number of features extracted by the CNN
'''
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


class Configs:
    def __init__(self):
        # General parameters
        self.task_name = 'regression'  # or 'short_term_forecast', 'imputation', 'anomaly_detection', 'regression'
        self.seq_len = 32  # Input sequence length
        self.label_len = 48  # Label length for forecasting
        self.pred_len = 24  # Prediction length
        self.e_layers = 2  # Number of encoder layers
        self.d_model = 512  # Dimension of the model
        self.embed = 'timeF'  # Embedding type, e.g., 'timeF' for time features
        self.freq = 's'  # Frequency of the data, e.g., 'h' for hourly data
        self.dropout = 0.1  # Dropout rate
        self.top_k = 1
        self.d_ff = 512  # Dimension of the feedforward network model
        self.num_kernels = 6 # Number of kernels in the Inception block
        self.enc_in = 32  # Number of input features (for forecasting and imputation)
        self.output_attention = False  # Whether to output attention weights
        self.factor = 1 # 'attn factor'
        self.n_heads = 8  # Number of heads in the multi-head attention
        self.activation = 'gelu'  # Activation function


class PITN(nn.Module):
    def __init__(self, configs):
        super(PITN, self).__init__()
        self.configs = configs
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.model = nn.ModuleList([TemporalBlock(configs) for _ in range(configs.e_layers)])
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq,
                                           configs.dropout)
        self.layer = configs.e_layers
        self.layer_norm = nn.LayerNorm(configs.d_model)
        # add layer_norm_adv for adversarial input
        self.layer_norm_adv = nn.LayerNorm(configs.d_model)
        self.act = F.gelu
        self.dropout = nn.Dropout(configs.dropout)
        self.projection = nn.Linear(512, 64)  # Adjust the output dimension for regression
        self.decision = nn.Linear(67, 1)
        
    def regression(self, X, adv_flag=False):
        # embedding
        x_enc, feat_1, feat_2, feat_3 = X[:, :32], X[:, 32], X[:, 33], X[:, 34]

        x_enc = x_enc.unsqueeze(1)
        feat_1 = feat_1.unsqueeze(1)
        feat_2 = feat_2.unsqueeze(1)
        feat_3 = feat_3.unsqueeze(1)

        enc_out = self.enc_embedding(x_enc)  # [B, T, C]
        # TimesNet
        for i in range(self.layer):
            if adv_flag:
                enc_out = self.layer_norm_adv(self.model[i](enc_out))
            else:
                enc_out = self.layer_norm(self.model[i](enc_out))
        # Output
        # the output transformer encoder/decoder embeddings don't include non-linearity
        output = self.act(enc_out)
        output = self.dropout(output)
        # (batch_size, seq_length * d_model)
        output = output.reshape(output.shape[0], -1)
        hidden_feature = output
        output = self.projection(output)  # (batch_size, 64)
        output = torch.cat((output, feat_1, feat_2, feat_3), 1)
        output = self.decision(output)
        return output, hidden_feature

    def regression_net(self, x_enc, feat_1, feat_2, feat_3):
        # embedding
        return self.regression(torch.cat((x_enc, feat_1, feat_2, feat_3), 1))

    def Physics_net(self, feature, feat_1, feat_2, feat_3):
        # No need to calculate the gradient of adversarial loss
        u, _ = self.regression_net(feature, feat_1, feat_2, feat_3)
        u_feat_1 = torch.autograd.grad(u, feat_1,
                                       grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_feat_2 = torch.autograd.grad(u, feat_2,
                                       grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_feat_3 = torch.autograd.grad(u, feat_3,
                                       grad_outputs=torch.ones_like(u), create_graph=True)[0]
        pred_physics = (u[:-1, 0]
                        + (u_feat_1[:-1, 0] * (feat_1[1:, 0] - feat_1[:-1, 0]))
                        + (u_feat_2[:-1, 0] * (feat_2[1:, 0] - feat_2[:-1, 0]))
                        + (u_feat_3[:-1, 0] * (feat_3[1:, 0] - feat_3[:-1, 0]))
                        )
        return u, pred_physics

    def forward(self, X, flag, adv_flag):
        # flag to show calculate physics loss or not
        self.feature = X[:, :32].clone().detach().requires_grad_(True)
        self.feat_1 = X[:, 32].clone().detach().requires_grad_(True).unsqueeze(1)
        self.feat_2 = X[:, 33].clone().detach().requires_grad_(True).unsqueeze(1)
        self.feat_3 = X[:, 34].clone().detach().requires_grad_(True).unsqueeze(1)

        if flag:
            u, pred_physics = self.Physics_net(self.feature, self.feat_1, self.feat_2, self.feat_3)
            return u, pred_physics
        else:
            u, hidden_feature = self.regression(X)
            return u, hidden_feature

    def function(self, X, train_out):
        u, _ = self.regression(X, adv_flag=True)
        pred_loss = torch.mean(torch.square(u - train_out))
        return pred_loss



### Import a Demo Data

In [2]:
# load an example data for demo
import pandas as pd
df_demo_data = pd.read_pickle('data_demo.pkl')

### Preprocess and Prepare the Train/Test Datasets

In [3]:
import numpy as np
from sklearn import preprocessing
import random
import torch.utils.data as Data
import warnings
# Initialize a SEED value to ensure that the random processes in the code can be reproduced.
SEED = 123

# Call the function with seed value
warnings.filterwarnings("ignore", category=DeprecationWarning)


# The keys for the beat data (beat_key), the target (out_key), and the features (feat_keys) are defined
beat_key = 'bioz_beats'
out_key = 'DBP'
shift = 2.0
beta = 1
feat_keys = ['phys_feat_1','phys_feat_2','phys_feat_3']

# Data scaling of BP, input beats, and input features
# This scaler standardizes by removing the mean and scaling to unit variance
# This is done to ensure having the same scale, which can improve the performance of machine learning algorithms
scaler_out = preprocessing.StandardScaler().fit(df_demo_data[out_key].to_numpy()[:, None])
mean_value = scaler_out.mean_
std_value = scaler_out.scale_

contra_shift = (shift - mean_value) / (std_value ** 2)
scaler_beats = preprocessing.StandardScaler().fit(np.concatenate(df_demo_data[beat_key].to_numpy())[:, None])
scaler_X = [preprocessing.StandardScaler().fit(df_demo_data[a].to_numpy()[:, None]) for a in feat_keys]

# Apply Scaling
# The scaled versions of the BP, input beats, and input features are then added to the dataframe
df_demo_data.loc[df_demo_data.index, beat_key + '_scaled'] = df_demo_data.apply(
    lambda x: np.concatenate(scaler_beats.transform(x[beat_key][:, None])), axis=1).to_numpy()

def scale_output(row):
    output = np.array([row[out_key]])[:, None]
    output = output.reshape(-1, 1)
    scaled_output = scaler_out.transform(output)
    return np.concatenate(scaled_output)[0]

df_demo_data[out_key + '_scaled'] = df_demo_data.apply(scale_output, axis=1).to_numpy()

# df_demo_data.loc[df_demo_data.index, out_key + '_scaled'] = df_demo_data.apply(
#     lambda x: np.concatenate(scaler_out.transform(np.array([x[out_key]])[:, None]))[0], axis=1).to_numpy()

def transform_scaler_X(x, tmp_key, tmp_count):
    value = np.array([x[tmp_key]])[:, None]
    value = value.reshape(-1, 1)
    scaled_value = scaler_X[tmp_count].transform(value)
    concatenated_value = np.concatenate(scaled_value)
    return concatenated_value

for tmp_key, tmp_count in zip(feat_keys, range(len(feat_keys))):
    # df_demo_data.loc[df_demo_data.index, tmp_key + '_scaled'] = df_demo_data.apply(
    #     lambda x: np.concatenate(scaler_X[tmp_count].transform(np.array([x[tmp_key]])[:, None])), axis=1).to_numpy()
    df_demo_data[tmp_key + '_scaled'] = df_demo_data.apply(
        lambda x, key=tmp_key, count=tmp_count: transform_scaler_X(x, key, count), axis=1).to_numpy()
# Fetch scaled feature names
X_keys = [a + '_scaled' for a in feat_keys]

# Prepare train/test using minimal training the BP
# Fetch data shapes
length_seq_x = df_demo_data.apply(lambda x: len(x[beat_key + '_scaled']), axis=1).unique()[0]

# Set the length of the target to 1
length_seq_y = 1

# Start with all points
# Reshape the scaled beat data into a 2D array where each row corresponds to a sample and each column corresponds to a time point in the beat sequence
# The same is done for the features and the target
all_beats = np.reshape(np.concatenate(df_demo_data[beat_key + '_scaled'].values), (len(df_demo_data), length_seq_x))
[all_feat1, all_feat2, all_feat3] = [df_demo_data[a].values[:, None] for a in X_keys]
all_out = df_demo_data[out_key + '_scaled'].values[:, None]

# Used only for plotting purposes
out_max_rescaled = np.concatenate(scaler_out.inverse_transform(all_out[:, 0][:, None])).max()
out_min_rescaled = np.concatenate(scaler_out.inverse_transform(all_out[:, 0][:, None])).min()
# Given different trials have time gaps, ignore first 3 instances from indices to prevent discontiunity in training
list_all_length = [0]
for _, df_tmp in df_demo_data.groupby(['trial_id']):
    list_all_length.append(len(df_tmp))
ix_ignore_all = np.concatenate(np.array([np.arange(a, a + 3, 1) for a in list(np.cumsum(list_all_length)[:-1])]))
# Update the final indices set
ix_all = list(set(np.arange(len(df_demo_data))) - set(ix_ignore_all))

# Separate train/test based on minimal training criterion
random.seed(0)
bp_dist = df_demo_data[out_key].values

# Find indices for train and test datasets
# The target values are sorted in ascending order, and the sorted indices are split into multiple subsets
# For each subset, a random index is selected as a training index
ix_split = np.split([a for a in np.argsort(bp_dist) if a not in set(ix_ignore_all)], np.cumsum(
    np.histogram(bp_dist[ix_all], bins=np.arange(bp_dist[ix_all].min(), bp_dist[ix_all].max(), 1))[0]))
ix_train = [random.Random(4).choice(a) if len(a) > 0 else -1 for a in ix_split]
ix_train = list(set(ix_train) - set([-1]))
# Test set is all remaining points not used for training
ix_test = list(set(ix_all) - set(ix_train))

 # Build train and test datasets based on the indices
train_beats = all_beats[ix_train, :]
test_beats = all_beats[ix_test, :]
[train_feat1, train_feat2, train_feat3] = [all_feat1[ix_train, :], all_feat2[ix_train, :], all_feat3[ix_train, :]]
[test_feat1, test_feat2, test_feat3] = [all_feat1[ix_test, :], all_feat2[ix_test, :], all_feat3[ix_test, :]]
train_out = all_out[ix_train, :]
test_out = all_out[ix_test, :]
train_out = [float(item) for item in train_out]
test_out = [float(item) for item in test_out]
train_out = torch.tensor(train_out, dtype=torch.float32)
test_out = torch.tensor(test_out, dtype=torch.float32)

train_feat1 = [float(item) for item in train_feat1]
train_feat2 = [float(item) for item in train_feat2]
train_feat3 = [float(item) for item in train_feat3]


In [4]:

#### Define model input tensors
# The training, testing, and all data are converted to TensorFlow tensors
# The tensors for the different datasets are grouped into lists 
model_inp = torch.tensor(train_beats, dtype=torch.float32)
feat1_inp = torch.tensor(train_feat1, dtype=torch.float32)
feat2_inp = torch.tensor(train_feat2, dtype=torch.float32)
feat3_inp = torch.tensor(train_feat3, dtype=torch.float32)

inp_feat1 = feat1_inp.unsqueeze(1)
inp_feat2 = feat2_inp.unsqueeze(1)
inp_feat3 = feat3_inp.unsqueeze(1)
# train_out 在计算MSE时发生错误，维度不匹配！！！！！！导致了计算MSE有问题
# 原本train_out的维度应该是（n,1），但是实际的维度是(n, )，所以需要增加一个维度
train_out = train_out.unsqueeze(1)
test_out = test_out.unsqueeze(1)


inp_comb = torch.cat((model_inp, inp_feat1, inp_feat2, inp_feat3), dim=1)
input_comb_np = inp_comb.clone().detach().numpy()

train_dataset = Data.TensorDataset(inp_comb, train_out)
train_data_iter = Data.DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

test_feat1 = [float(item) for item in test_feat1]
test_feat2 = [float(item) for item in test_feat2]
test_feat3 = [float(item) for item in test_feat3]

model_inp_test = torch.tensor(test_beats, dtype=torch.float32)
feat1_inp_test = torch.tensor(test_feat1, dtype=torch.float32)
feat2_inp_test = torch.tensor(test_feat2, dtype=torch.float32)
feat3_inp_test = torch.tensor(test_feat3, dtype=torch.float32)

feat1_inp_test = feat1_inp_test.unsqueeze(1)
feat2_inp_test = feat2_inp_test.unsqueeze(1)
feat3_inp_test = feat3_inp_test.unsqueeze(1)

inp_comb_test = torch.cat((model_inp_test, feat1_inp_test, feat2_inp_test, feat3_inp_test), dim=1)

all_feat1 = [float(item) for item in all_feat1]
all_feat2 = [float(item) for item in all_feat2]
all_feat3 = [float(item) for item in all_feat3]

model_inp_all = torch.tensor(all_beats, dtype=torch.float32)
feat1_inp_all = torch.tensor(all_feat1, dtype=torch.float32)
feat2_inp_all = torch.tensor(all_feat2, dtype=torch.float32)
feat3_inp_all = torch.tensor(all_feat3, dtype=torch.float32)

feat1_inp_all = feat1_inp_all.unsqueeze(1)
feat2_inp_all = feat2_inp_all.unsqueeze(1)
feat3_inp_all = feat3_inp_all.unsqueeze(1)

inp_comb_all = torch.cat((model_inp_all, feat1_inp_all, feat2_inp_all, feat3_inp_all), dim=1)
ix_all = torch.tensor(ix_all)
ix_train = torch.tensor(ix_train)
ix_test = torch.tensor(ix_test)

test_dataset = Data.TensorDataset(inp_comb_test, test_out)
test_data_iter = Data.DataLoader(test_dataset, batch_size=len(inp_comb), drop_last=False)

### Train PITN model

In [5]:
configs = Configs()
model_PITN = PITN(configs)
import time

# A Deep Neural Network model is initialized with the dimension of the beats, the diemnsion of each feature, and the number of neurons in the first dense layer
N0 = 200

x_train, x_boundary, u_boundary = training_data_latin_hypercube(inp_comb, train_out, N_inner=N0)
x_adv = np.array([]).reshape((0, 35))
x_train = np.vstack([x_train, x_adv])
# No need to retrain, directly generate adversarial samples
x_adv = generate_attack_samples(model_PITN, device, x_train, N0, train_out)
# using diffusion to generate adversarial samples
foward_diffusion = diffusion_foward()
x_diffusion = foward_diffusion.q_sample(x_train, torch.tensor([20]))

# Two lists are initialized to keep track of the training and testing loss during each epoch
loss_list_pinn = []
test_loss_list_pinn = []
loss_total_epoch = []
loss_physics_epoch = []
loss_dnn_epoch = []
criterion = MultiPosConLoss()
print("PITN model training started")
optimizer = optim.Adam(model_PITN.parameters(), lr=10e-4)

best_loss = float("inf")
# Two lists are initialized to keep track of the training and testing loss during each epoch
epochs = 2000
inp_comb_adv = x_adv
for epoch in range(epochs):
    start = time.time()
    train_l_sum = 0.0
    optimizer.zero_grad()
    # Traditional out
    model_PITN.to(device=device)
    # train on clean data
    # if inp_comb and train_out are not tensor, then turn them into tensor
    if not torch.is_tensor(inp_comb):
        inp_comb = torch.tensor(inp_comb, dtype=torch.float32)
    if not torch.is_tensor(train_out):
        train_out = torch.tensor(train_out, dtype=torch.float32)
    inp_comb, train_out = inp_comb.to(device=device), train_out.to(device=device)
    # model_dnn_pinn的参数意思是，输入数据，是否计算physics loss，是否使用auxiliary BN
    output, feature_clean = model_PITN(inp_comb, False, False)
    loss_dnn_clean = output - train_out
    loss_dnn_clean = torch.mean(torch.square(loss_dnn_clean))

    if not torch.is_tensor(inp_comb_adv):
        inp_comb_adv = torch.tensor(inp_comb_adv, dtype=torch.float32)
    inp_comb_adv = inp_comb_adv.to(device=device)
    output, feature_adv = model_PITN(inp_comb_adv, False, True)
    # concat train_out by twice, and then use it as labels
    adv_labels = train_out
    loss_dnn_adv = output - adv_labels
    loss_dnn_adv = torch.mean(torch.square(loss_dnn_adv))

    # using contrastive learning
    bsz = len(feature_clean)
    features = torch.cat((feature_clean, feature_adv), dim=0)
    label = 0
    labels = np.zeros([1, bsz])
    mask = [True] * bsz
    for i in range(bsz):
        if mask[i]:  # if the sample is not used
            if i == bsz - 1:
                labels[0][i] = label
                mask[i] = False
                break
            else:
                labels[0][i] = label
                for j in range(i + 1, bsz):
                    if abs((train_out[i] - train_out[j]).item()) <= abs(contra_shift):
                        labels[0][j] = label
                        mask[j] = False
            label += 1
            mask[i] = False
    # concat more if samples are used multiple times
    labels = np.concatenate((labels, labels), axis=1)
    labels = torch.tensor(labels, dtype=torch.int).to(device=device)
    loss_con = criterion({'feats': features, 'labels': labels})

    # train on adversarial data and contrastive learning
    loss_dnn = loss_dnn_clean + loss_dnn_adv + loss_con

    # train on adversarial data
    loss_dnn = loss_dnn_clean + loss_dnn_adv

    # train on only clean data
    # loss_dnn = loss_dnn_clean

    # Physics loss
    inp_comb_all = inp_comb_all.to(device=device)
    y, pred_physics = model_PITN(inp_comb_all, True, False)
    physics_loss_ini = pred_physics - y[1:, 0]
    physics_loss = torch.mean(torch.square(physics_loss_ini[ix_all[:-1].view(-1, 1)]))

    # physics_loss = K.mean(K.square(tf.gather_nd(physics_loss_ini, indices=np.array(ix_all[:-1])[:, None])))
    loss_total = loss_dnn + physics_loss * beta
    loss_total_epoch.append(np.round(loss_total.cpu().detach().numpy(), 4))
    loss_physics_epoch.append(np.round(physics_loss.cpu().detach().numpy(), 4))
    loss_dnn_epoch.append(np.round(loss_dnn.cpu().detach().numpy(), 4))

    loss_total.backward()
    optimizer.step()
    train_l_sum += loss_total.item()
    loss_list_pinn.append(float(loss_dnn.item()))
    loss_final = np.min(loss_list_pinn)

    # model evaluation
    inp_comb_test, test_out = inp_comb_test.to(device=device), test_out.to(device=device)
    pred_out, _ = model_PITN(inp_comb_test, False, False)
    test_loss = pred_out - test_out
    test_loss = torch.mean(torch.square(test_loss))
    test_loss_list_pinn.append(float(test_loss))
    end = time.time()
    # print every 100 epochs
    if epoch % 100 == 0:
        print(
            "epoch: {}, time: {}, loss_dnn:{}, loss_physics: {}, test_loss: {}".format(epoch, round(end - start, 2),
                                                                                       np.round(loss_final, 4),
                                                                                       np.round(
                                                                                           physics_loss.cpu().detach().numpy(),
                                                                                           4), test_loss))

    if test_loss < best_loss:
        best_loss = test_loss

        torch.save(model_PITN.state_dict(), "best.pth")

    # if epoch == epochs - 1:
    if (loss_final <= 0.01) | (epoch == epochs - 1):
        # wandb.finish()
        torch.save(model_PITN.state_dict(),
                   "last.pth")

        print("PITN model training Completed. Epoch %d/%d -- loss: %.4f" % (
            epoch, epochs, np.round(loss_total.cpu().detach().numpy(), 4)))
        



PITN model training started
epoch: 0, time: 35.45, loss_dnn:5.6148, loss_physics: 0.03009999915957451, test_loss: 1.5334891080856323


KeyboardInterrupt: 

### Train the PITN model

In [28]:
#The trained model's predictions on the test dataset are computed
pred_out = PITN(inp_comb_test)

#The Pearson correlation coefficient and the Root Mean Square Error are calculated between the actual and predicted test outcomes
corr_conv = np.corrcoef(np.concatenate(test_out)[:], np.concatenate(pred_out)[:])[0][1]
rmse_conv = np.sqrt(np.mean(np.square
                           (np.concatenate(scaler_out.inverse_transform(np.concatenate(test_out)[:][:, None]))-
                            np.concatenate(scaler_out.inverse_transform(np.concatenate(pred_out)[:][:, None])))))

pred_out = PITN(inp_comb_test)
corr_pinn = np.corrcoef(np.concatenate(test_out)[:], np.concatenate(pred_out)[:])[0][1]
rmse_pinn = np.sqrt(np.mean(np.square(
    np.concatenate(scaler_out.inverse_transform(np.concatenate(test_out)[:][:, None]))-
    np.concatenate(scaler_out.inverse_transform(np.concatenate(pred_out)[:][:, None])))))

print('#### PITN Performance ####')
print('Corr: %.2f,  RMSE: %.1f'%(corr_pinn, rmse_pinn))

AttributeError: 'Tensor' object has no attribute 'task_name'